<a href="https://colab.research.google.com/github/wyxData/afconEDA/blob/main/afcon2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using latest data, I will rank AFCON countries based on individual player strength and create powerBi Dashboard based on the cleaned data set. I stored data from Kaggle to github for ease of access

In [45]:
#Load libraries and read data
import pandas as pd
fifa_df = pd.read_csv('https://raw.githubusercontent.com/wyxData/afconEDA/main/male_players.csv')

<ipython-input-45-95d744db49e6>:3: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_df = pd.read_csv('https://raw.githubusercontent.com/wyxData/afconEDA/main/male_players.csv')


In [46]:
#Explore Data for correctiness
fifa_df.head()

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3


In [104]:
fifa_df.shape

(180021, 109)

Key information required include

*   Player Name
*   Nationality
*   Nation Team Position
*   International Reputation
*   Rating
*   Potential
*   Work Rate








In [105]:
#Print all columns to identify what to drop:
for col in fifa_df.columns:
    print(col)

player_id
player_url
fifa_version
fifa_update
update_as_of
short_name
long_name
player_positions
overall
potential
value_eur
wage_eur
age
dob
height_cm
weight_kg
club_team_id
club_name
league_id
league_name
league_level
club_position
club_jersey_number
club_loaned_from
club_joined_date
club_contract_valid_until_year
nationality_id
nationality_name
nation_team_id
nation_position
nation_jersey_number
preferred_foot
weak_foot
skill_moves
international_reputation
work_rate
body_type
real_face
release_clause_eur
player_tags
player_traits
pace
shooting
passing
dribbling
defending
physic
attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys
skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control
movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance
power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots
mentality_aggression
mentality_interceptions
m

In [47]:
fifa_df= fifa_df[['player_id','short_name','long_name', 'nationality_name', 'club_position','nation_position', 'overall', 'potential',  'international_reputation', 'work_rate']]
fifa_df.head()

,player_id,short_name,long_name,nationality_name,club_position,nation_position,overall,potential,international_reputation,work_rate
0,231747,K. Mbappé,Kylian Mbappé Lottin,France,LW,LW,91,94,5,High/Low
1,239085,E. Haaland,Erling Braut Haaland,Norway,ST,ST,91,94,5,High/Medium
2,192985,K. De Bruyne,Kevin De Bruyne,Belgium,SUB,CAM,91,91,5,High/Medium
3,158023,L. Messi,Lionel Andrés Messi Cuccittini,Argentina,RF,RW,90,90,5,Low/Low
4,165153,K. Benzema,Karim Benzema,France,RS,NaN,90,90,5,Medium/Medium


Obtain AFCON teama and player list. Scrap data from WIkipedia, DSTV, Goal?

In [4]:
#Scrap Data from Supersports Site
import requests
from bs4 import BeautifulSoup
response = requests.get('https://supersport.com/football/africa-cup-of-nations/logs')
team_soup = BeautifulSoup(response.text, 'html')
required_sections= team_soup.find_all('table', class_="flex flex-col")

In [163]:
#Create Dataframe for AFCON Nation and Group pairings
nation = []
group = []
for section in required_sections:
    group_name = section.find('th', class_="default-table-text flex flex-5 flex-row items-center text-left").text.strip()
    countries = section.find_all('span', class_="default-table-text hidden justify-self-start overflow-hidden text-ellipsis whitespace-nowrap md:block")
    country_names = [country.text.strip() for country in countries]
    # Append data to lists
    group.extend([group_name] * len(country_names))
    nation.extend(country_names)
# Create a pandas DataFrame
afcon_list_df = pd.DataFrame({ "nationality_name": nation, "group": group})
print(afcon_list_df)


     nationality_name    group
0       Cote d'Ivoire  Group A
1   Equatorial Guinea  Group A
2       Guinea-Bissau  Group A
3             Nigeria  Group A
4          Cape Verde  Group B
5               Egypt  Group B
6               Ghana  Group B
7          Mozambique  Group B
8            Cameroon  Group C
9              Gambia  Group C
10             Guinea  Group C
11            Senegal  Group C
12            Algeria  Group D
13             Angola  Group D
14       Burkina Faso  Group D
15         Mauritania  Group D
16               Mali  Group E
17            Namibia  Group E
18       South Africa  Group E
19            Tunisia  Group E
20           DR Congo  Group F
21            Morocco  Group F
22           Tanzania  Group F
23             Zambia  Group F


In [43]:
#Extract AFCON squad from wikipedia
response = requests.get('https://en.wikipedia.org/wiki/2023_Africa_Cup_of_Nations_squads')
players_soup = BeautifulSoup(response.text, 'html')
required_data= players_soup.find_all('tr', class_="nat-fs-player")

positions = []
player_names = []
for data in required_data:
    td_elements = data.find_all('td')
    if len(td_elements) > 1:
        position = td_elements[1].find('a').text.strip()
        positions.append(position)
        th_element = data.find('th')
        if th_element:
            player_name = th_element.find('a').text.strip()
            player_names.append(player_name)
afcon_squad_df = pd.DataFrame(list(zip(positions, player_names)), columns=['position', 'player_name'])
afcon_squad_df.head()

,position,player_name
0,GK,Yahia Fofana
1,DF,Ousmane Diomande
2,DF,Ghislain Konan
3,MF,Jean Michaël Seri
4,DF,Wilfried Singo


In [55]:
#filter afcon squad from fifa list
player_filter_list = afcon_squad_df['player_name']
confirmed_afcon_list= fifa_df[fifa_df['long_name'].isin(player_filter_list)]
confirmed_afcon_list.tail()

,player_id,short_name,long_name,nationality_name,club_position,nation_position,overall,potential,international_reputation,work_rate
176529,218880,S. Marreh,Sulayman Marreh,Gambia,RES,NaN,58,67,1,Medium/High
176581,221556,O. Mfulu,Omenuke Mfulu,Congo,RES,NaN,58,66,1,Medium/Medium
177002,216485,A. Tshibola,Aaron Tshibola,England,RES,NaN,57,75,1,Medium/Medium
177251,209840,J. Jeanvier,Julian Jeanvier,France,SUB,NaN,57,64,1,Medium/Medium
177889,202014,R. Lopes,Roberto Lopes,Republic of Ireland,RCB,NaN,56,61,1,Medium/Medium


In [159]:
#First we standardize Country Names in FIFA dataset (Ivory coast, capeverde and COngo)
#replacement_names= ["Cote d'Ivoire",'DR Congo','Cape Verde']
fifa_df['nationality_name']= fifa_df['nationality_name'].str.replace("Côte","Cote").str.replace("Congo DR","DR Congo").str.replace("Cape Verde Islands","Cape Verde").str.replace("Guinea Bissau","Guinea-Bissau")


In [160]:
#Filter AFCON COuntries from FIFA Dataset
filter_list = afcon_list_df['nation']
afcon_player_list= fifa_df[fifa_df['nationality_name'].isin(filter_list)]
afcon_player_list.shape

(10584, 10)

In [161]:
afcon_player_list['nationality_name'].nunique()

24

In [165]:
#Merge to append group information
merged_afon_list = pd.merge(afcon_player_list,afcon_list_df, on= 'nationality_name')
merged_afon_list.head()

,player_id,short_name,long_name,nationality_name,club_position,nation_position,overall,potential,international_reputation,work_rate,group
0,209331,M. Salah,Mohamed Salah Ghaly,Egypt,RW,NaN,89,89,5,High/Medium,Group B
1,210648,A. Hegazi,Ahmed Elsayed Ali Elsayed Hegazy,Egypt,RES,NaN,78,78,2,Low/High,Group B
2,226078,Trezeguet,Mahmoud Ahmed Ibrahim Hassan,Egypt,LM,NaN,76,76,1,High/Medium,Group B
3,256675,O. Marmoush,Omar Khaled Mohamed Marmoush,Egypt,ST,NaN,75,80,1,High/Low,Group B
4,211454,M. Elneny,Mohamed Naser Elsayed Elneny,Egypt,RES,NaN,75,75,1,Medium/High,Group B


In [171]:
#Assumption1. Country Stregth based on avarage of overall player strength.
merged_afon_list.groupby('nationality_name')['overall'].mean()

nationality_name
Algeria              70.174242
Angola               68.020000
Burkina Faso         67.116959
Cameroon             67.926768
Cape Verde           68.552174
Cote d'Ivoire        68.295635
DR Congo             67.609615
Egypt                69.542969
Equatorial Guinea    66.951613
Gambia               65.123853
Ghana                66.673401
Guinea               67.245902
Guinea-Bissau        66.290698
Mali                 67.748560
Mauritania           64.375000
Morocco              69.419393
Mozambique           70.638298
Namibia              67.315789
Nigeria              67.191851
Senegal              67.831981
South Africa         67.068017
Tanzania             64.904762
Tunisia              67.786184
Zambia               67.080000
Name: overall, dtype: float64